In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
training = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv')
training['train_test'] = 1
test['train_test'] = 0
test['failure'] = np.NaN
all_data = pd.concat([training,test]) 
all_data = all_data.dropna()
all_data = all_data.drop(['id'], axis=1)

In [ ]:
all_data

In [ ]:
#one-hot encoding
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

attribute_names = all_data.columns.tolist()
print(attribute_names)


In [ ]:
all_dummies = pd.get_dummies(all_data[attribute_names])
X_train = all_dummies[all_data.train_test == 1].drop(['train_test'], axis=1)
X_test = all_dummies[all_data.train_test == 0].drop(['train_test'], axis=1)

In [ ]:
X_atts = X_train.drop(['loading', 'failure', 'product_code_A',
                                      'product_code_B', 'product_code_C', 'product_code_D', 
                                      'product_code_E', 'attribute_0_material_5', 'attribute_0_material_7', 
                                      'attribute_1_material_5', 'attribute_1_material_6', 'attribute_1_material_8'],axis=1)
all_attributes = X_atts.columns.tolist()

In [ ]:
#feature engineering
#since the correlation of the data is very low almost everywhere and the names are unspecific,
#we cant really drop anything so I will just do some basic normalization

from sklearn.preprocessing import StandardScaler, MinMaxScaler
'''columns_to_normalize = ['measurement_0', 'measurement_1', 'measurement_2', 
                        'measurement_3', 'measurement_4', 'measurement_5', 
                        'measurement_6', 'measurement_7', 'measurement_8', 
                        'measurement_9', 'measurement_10', 'measurement_11', 
                        'measurement_12', 'measurement_13', 'measurement_14', 
                        'measurement_15', 'measurement_16', 'measurement_17']'''
columns_to_normalize = all_attributes
scaler = MinMaxScaler()
X_train[columns_to_normalize] = scaler.fit_transform(X_train[columns_to_normalize])


In [ ]:
X_train

In [ ]:
plt.figure(figsize=(20, 16))
check_A = X_train.drop(['product_code_B', 'product_code_C', 'product_code_D', 'product_code_E'], axis=1)
sb.heatmap(check_A.corr(), annot=False, cmap='coolwarm')

plt.title('Correlation Heatmap')
plt.show()

In [ ]:
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
#from sklearn.model_selection import KFold

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import VotingClassifier

import wandb
#wandb.init(project='KaggleAugust2022', name='August2022')